In [25]:
import pandas as pd
import polars as pl
import toml
from pathlib import Path
from sqlalchemy import create_engine

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.1%}'.format)

In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]
output_dir = output_path / 'RTP/mode_share'

# Write out CSVs with a numeric value prefix to help organize outputs 
incr = 1

In [50]:
def prep_trip_df(trip):
    df_trip = trip.copy()
    df_trip.rename(columns={'mode':'Mode'}, inplace=True)
    df_trip['Trip Type'] = 'Non-Work'
    df_trip.loc[df_trip['dpurp'] == 'Work', 'Trip Type'] = 'Work'

    # add all tripa
    df = df_trip.copy()
    df['Trip Type'] = 'All Trips'
    df_trip = pd.concat([df_trip.sort_values('Trip Type'), df], ignore_index=True)

    return df_trip

In [76]:
trip_county = prep_trip_df(pd.read_csv(output_path / 'agg/dash/mode_share_county.csv'))
trip_rg = prep_trip_df(pd.read_csv(output_path / 'agg/dash/mode_share_rg.csv'))

trip_rgc = pd.read_csv(output_path / 'agg/dash/mode_share_rgc.csv')
# add all RGCs
df = trip_rgc[trip_rgc['hh_rgc'] != 'Not in RGC'].groupby(['dpurp','mode']).sum()[['trexpfac']].reset_index()
df['hh_rgc'] = 'All RGCs'
trip_rgc = pd.concat([df, trip_rgc])
trip_rgc = prep_trip_df(trip_rgc)

person = pd.read_csv(output_path / 'agg/dash/person_geog.csv')

### Region

In [52]:
df1 = trip_county.groupby(['Trip Type','Mode'],as_index=False)['trexpfac'].sum()
df2 = trip_county.groupby(['Trip Type'],as_index=False)['trexpfac'].sum()
df2.rename(columns={'trexpfac':'total_trips'}, inplace=True)

df = df1.merge(df2, on='Trip Type')
df['mode_share'] = df['trexpfac'] / df['total_trips']

df = df.pivot_table(columns='Trip Type', index='Mode', values='mode_share')

f_name = str(incr)+'_mode_share_region.csv'
df.to_csv(output_dir / f_name); incr+=1

df

Trip Type,All Trips,Non-Work,Work
Mode,,,
Bike,1.5%,1.5%,1.4%
HOV2,21.4%,23.2%,9.4%
HOV3+,14.1%,15.4%,5.9%
SOV,44.4%,40.3%,71.3%
School Bus,1.6%,1.8%,0.0%
TNC,0.3%,0.3%,0.6%
Transit,2.4%,2.4%,2.6%
Walk,14.3%,15.2%,8.9%


### County

In [64]:
def calc_mode_share(df_trip, geog):
    
    # num trips by mode
    df1 = df_trip.groupby([geog, 'Trip Type', 'Mode'],as_index=False)['trexpfac'].sum()
    # num trips
    df2 = df_trip.groupby([geog, 'Trip Type'],as_index=False)['trexpfac'].sum()
    df2.rename(columns={'trexpfac':'total_trips'}, inplace=True)

    df = df1.merge(df2, on=[geog, 'Trip Type'])
    # mode share
    df['mode_share'] = df['trexpfac'] / df['total_trips']

    # mode share by trip type
    df_all = df.loc[df['Trip Type']=="All Trips"].pivot_table(columns='Mode', index=geog, values='mode_share')
    df_work = df.loc[df['Trip Type']=="Work"].pivot_table(columns='Mode', index=geog, values='mode_share')
    df_non_work = df.loc[df['Trip Type']=="Non-Work"].pivot_table(columns='Mode', index=geog, values='mode_share')

    return df_all, df_work, df_non_work


df_all, df_work, df_non_work = calc_mode_share(trip_county,'hh_county')

In [65]:
f_name = str(incr)+'_mode_share_county.csv'
df_all.to_csv(output_dir / f_name); incr+=1

df_all

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_county,,,,,,,,
King,1.6%,20.3%,13.2%,43.4%,1.5%,0.4%,3.4%,16.2%
Kitsap,1.2%,22.2%,14.2%,44.6%,1.5%,0.2%,1.8%,14.3%
Outside Region,NaN,27.3%,27.3%,45.5%,NaN,NaN,NaN,NaN
Pierce,1.4%,23.1%,15.5%,45.0%,1.9%,0.2%,0.9%,12.0%
Snohomish,1.3%,22.4%,15.2%,46.4%,1.7%,0.2%,1.4%,11.5%


In [66]:
f_name = str(incr)+'_mode_share_county_nonwork.csv'
df_non_work.to_csv(output_dir / f_name); incr+=1

df_non_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_county,,,,,,,,
King,1.6%,22.1%,14.4%,39.3%,1.7%,0.3%,3.5%,17.0%
Kitsap,1.2%,23.9%,15.3%,41.3%,1.7%,0.2%,1.2%,15.1%
Outside Region,NaN,28.6%,28.6%,42.9%,NaN,NaN,NaN,NaN
Pierce,1.4%,24.8%,16.8%,40.9%,2.2%,0.2%,0.8%,12.9%
Snohomish,1.3%,24.3%,16.5%,42.1%,1.9%,0.2%,1.3%,12.4%


In [67]:
f_name = str(incr)+'_mode_share_county_work.csv'
df_work.to_csv(output_dir / f_name); incr+=1

df_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_county,,,,,,,,
King,1.7%,9.1%,5.6%,68.6%,0.0%,0.8%,3.0%,11.3%
Kitsap,0.8%,9.4%,5.7%,70.5%,0.0%,0.2%,5.8%,7.6%
Outside Region,NaN,NaN,NaN,100.0%,NaN,NaN,NaN,NaN
Pierce,1.1%,9.9%,6.3%,75.8%,0.0%,0.3%,1.3%,5.4%
Snohomish,0.9%,9.8%,6.4%,75.7%,0.0%,0.3%,1.8%,5.1%


## Regional Growth Center

In [69]:
df_all, df_work, df_non_work = calc_mode_share(trip_rgc, 'hh_rgc')

f_name = str(incr)+'_mode_share_rgc.csv'
df_all.to_csv(output_dir / f_name); incr+=1

df_all

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rgc,,,,,,,,
All RGCs,3.0%,10.9%,5.3%,29.2%,0.5%,1.5%,6.7%,43.0%
Auburn,2.5%,19.2%,10.1%,38.1%,1.0%,0.4%,3.0%,25.8%
Bellevue,2.4%,11.4%,5.1%,32.0%,0.4%,0.8%,4.7%,43.1%
Bothell Canyon Park,1.0%,20.8%,11.0%,50.4%,1.8%,0.2%,1.3%,13.5%
Bremerton,2.3%,14.4%,7.2%,34.2%,0.5%,0.6%,3.7%,37.1%
Burien,1.7%,17.4%,9.2%,44.2%,0.7%,0.4%,3.2%,23.2%
Everett,1.8%,13.5%,7.3%,37.7%,0.7%,0.5%,2.4%,36.2%
Federal Way,1.6%,20.2%,15.4%,37.7%,2.0%,0.2%,1.7%,21.2%
Greater Downtown Kirkland,1.8%,18.2%,8.7%,47.2%,0.8%,0.4%,4.3%,18.8%


In [71]:
f_name = str(incr)+'_mode_share_rgc_nonwork.csv'
df_non_work.to_csv(output_dir / f_name); incr+=1

df_non_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rgc,,,,,,,,
All RGCs,3.0%,12.0%,5.8%,26.2%,0.6%,1.3%,7.1%,44.0%
Auburn,2.5%,20.5%,10.8%,34.4%,1.1%,0.3%,2.7%,27.6%
Bellevue,2.3%,12.5%,5.5%,28.5%,0.5%,0.7%,5.2%,44.8%
Bothell Canyon Park,0.9%,23.1%,12.2%,45.6%,2.1%,0.2%,1.1%,14.7%
Bremerton,2.3%,15.7%,7.7%,30.8%,0.6%,0.6%,2.5%,39.9%
Burien,1.6%,18.8%,9.8%,39.4%,0.8%,0.4%,3.4%,25.8%
Everett,1.7%,14.4%,7.9%,33.3%,0.8%,0.5%,2.5%,38.8%
Federal Way,1.5%,21.5%,16.4%,33.8%,2.2%,0.2%,1.9%,22.4%
Greater Downtown Kirkland,1.7%,19.8%,9.4%,43.0%,0.9%,0.3%,4.4%,20.4%


In [72]:
f_name = str(incr)+'_mode_share_rgc_work.csv'
df_work.to_csv(output_dir / f_name); incr+=1

df_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rgc,,,,,,,,
All RGCs,3.3%,5.5%,2.5%,43.6%,NaN,2.7%,4.5%,37.9%
Auburn,2.1%,9.3%,4.5%,65.7%,NaN,0.8%,5.5%,12.0%
Bellevue,3.1%,6.0%,2.6%,49.9%,NaN,1.4%,2.2%,34.7%
Bothell Canyon Park,1.3%,8.1%,4.8%,76.8%,NaN,NaN,2.0%,7.1%
Bremerton,2.3%,6.8%,3.7%,55.2%,NaN,0.7%,11.2%,20.1%
Burien,2.2%,9.5%,5.5%,72.3%,NaN,0.6%,2.0%,7.8%
Everett,2.1%,7.1%,3.3%,66.4%,NaN,0.7%,1.7%,18.7%
Federal Way,2.5%,8.6%,7.0%,71.2%,NaN,NaN,0.4%,10.3%
Greater Downtown Kirkland,2.1%,8.7%,4.3%,72.0%,NaN,0.5%,3.5%,8.9%


## Regional Geography

In [77]:
df_all, df_work, df_non_work = calc_mode_share(trip_rg, 'hh_rg_proposed')

In [79]:
f_name = str(incr)+'_mode_share_rg.csv'
df_all.to_csv(output_dir / f_name); incr+=1

df_all

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rg_proposed,,,,,,,,
CitiesTowns,1.2%,22.8%,16.5%,45.0%,2.1%,0.2%,0.7%,11.7%
Core,1.4%,22.3%,14.9%,45.6%,1.7%,0.2%,2.0%,12.0%
HCT,1.3%,22.6%,15.2%,45.8%,1.8%,0.2%,1.9%,11.3%
Metro,2.1%,17.8%,10.7%,40.1%,1.2%,0.6%,4.6%,22.9%
UU,1.2%,24.2%,17.7%,44.6%,2.1%,0.2%,0.7%,9.3%


In [80]:
f_name = str(incr)+'_mode_share_rg_nonwork.csv'
df_non_work.to_csv(output_dir / f_name); incr+=1

df_non_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rg_proposed,,,,,,,,
CitiesTowns,1.2%,24.6%,17.9%,40.6%,2.4%,0.2%,0.6%,12.6%
Core,1.4%,24.1%,16.2%,41.2%,2.0%,0.2%,1.9%,12.9%
HCT,1.3%,24.4%,16.5%,41.6%,2.0%,0.2%,1.8%,12.2%
Metro,2.0%,19.5%,11.7%,36.1%,1.4%,0.5%,4.7%,24.1%
UU,1.3%,26.1%,19.2%,40.4%,2.4%,0.1%,0.5%,10.0%


In [81]:
f_name = str(incr)+'_mode_share_rg_work.csv'
df_work.to_csv(output_dir / f_name); incr+=1

df_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rg_proposed,,,,,,,,
CitiesTowns,0.7%,10.3%,6.9%,76.2%,0.0%,0.2%,1.0%,4.8%
Core,1.2%,9.9%,6.4%,74.3%,0.0%,0.3%,2.2%,5.7%
HCT,1.0%,9.9%,6.4%,74.7%,0.0%,0.3%,2.6%,5.2%
Metro,2.3%,8.2%,4.7%,63.3%,0.0%,1.1%,3.9%,16.3%
UU,0.6%,10.3%,6.7%,76.8%,NaN,0.2%,1.4%,3.9%


## Equity Geography

In [ ]:
equity_geogs = summary_config['equity_geogs']

trip_equity_geog = pd.DataFrame()

for geog in equity_geogs:
    df = prep_trip_df(pd.read_csv(output_path/ ('agg/dash/mode_share_'+geog+'.csv')))
    df['equity_geog'] = df[geog].map({1:geog,
                                      0:"NOT in "+ geog})
    
    trip_equity_geog = pd.concat([trip_equity_geog, df[['dpurp', 'Mode', 'equity_geog', 'trexpfac', 'Trip Type']]])

df_all, df_work, df_non_work = calc_mode_share(trip_equity_geog, 'equity_geog')

In [96]:
f_name = str(incr)+'_mode_share_equity.csv'
df_all.to_csv(output_dir / f_name); incr+=1

df_all

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
equity_geog,,,,,,,,
NOT in hh_disability_reg,1.5%,21.4%,14.5%,44.8%,1.7%,0.3%,2.4%,13.4%
NOT in hh_elderly_reg,1.6%,20.9%,14.3%,43.3%,1.7%,0.3%,2.5%,15.2%
NOT in hh_english_reg,1.5%,21.1%,13.7%,44.6%,1.5%,0.3%,2.3%,14.9%
NOT in hh_poverty_50,1.5%,21.4%,14.1%,44.6%,1.6%,0.3%,2.3%,14.2%
NOT in hh_poverty_reg,1.4%,21.6%,14.3%,45.5%,1.6%,0.3%,2.1%,13.2%
NOT in hh_racial_50,1.4%,21.5%,14.0%,45.3%,1.6%,0.3%,2.2%,13.8%
NOT in hh_racial_reg,1.4%,21.5%,13.9%,45.9%,1.5%,0.3%,2.1%,13.4%
NOT in hh_youth_reg,1.7%,19.8%,12.0%,43.6%,1.2%,0.5%,3.3%,17.9%
hh_disability_reg,1.5%,21.3%,13.6%,43.9%,1.5%,0.4%,2.4%,15.5%


In [97]:
f_name = str(incr)+'_mode_share_equity_nonwork.csv'
df_non_work.to_csv(output_dir / f_name); incr+=1

df_non_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
equity_geog,,,,,,,,
NOT in hh_disability_reg,1.5%,23.3%,15.9%,40.5%,2.0%,0.2%,2.4%,14.2%
NOT in hh_elderly_reg,1.6%,22.8%,15.7%,39.0%,2.0%,0.3%,2.5%,16.1%
NOT in hh_english_reg,1.5%,23.0%,15.0%,40.6%,1.8%,0.3%,2.2%,15.7%
NOT in hh_poverty_50,1.5%,23.2%,15.4%,40.5%,1.9%,0.3%,2.3%,15.0%
NOT in hh_poverty_reg,1.4%,23.4%,15.6%,41.4%,1.9%,0.3%,2.1%,14.0%
NOT in hh_racial_50,1.4%,23.3%,15.2%,41.3%,1.8%,0.3%,2.1%,14.6%
NOT in hh_racial_reg,1.4%,23.3%,15.2%,41.8%,1.8%,0.2%,2.0%,14.3%
NOT in hh_youth_reg,1.7%,21.6%,13.1%,39.6%,1.4%,0.4%,3.3%,18.9%
hh_disability_reg,1.5%,23.0%,14.7%,40.0%,1.7%,0.3%,2.3%,16.4%


In [98]:
f_name = str(incr)+'_mode_share_equity_work.csv'
df_work.to_csv(output_dir / f_name); incr+=1

df_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
equity_geog,,,,,,,,
NOT in hh_disability_reg,1.3%,9.4%,5.9%,71.8%,0.0%,0.5%,2.6%,8.4%
NOT in hh_elderly_reg,1.5%,9.3%,5.8%,70.5%,0.0%,0.6%,2.6%,9.6%
NOT in hh_english_reg,1.4%,9.1%,5.6%,71.1%,0.0%,0.6%,2.8%,9.4%
NOT in hh_poverty_50,1.4%,9.4%,5.9%,71.5%,0.0%,0.5%,2.6%,8.8%
NOT in hh_poverty_reg,1.3%,9.4%,5.9%,72.2%,0.0%,0.5%,2.4%,8.3%
NOT in hh_racial_50,1.3%,9.4%,5.8%,72.5%,0.0%,0.5%,2.7%,8.0%
NOT in hh_racial_reg,1.2%,9.4%,5.8%,73.1%,0.0%,0.4%,2.7%,7.4%
NOT in hh_youth_reg,1.8%,8.8%,5.2%,67.8%,0.0%,0.8%,3.4%,12.2%
hh_disability_reg,1.4%,9.4%,5.8%,70.7%,0.0%,0.6%,2.6%,9.5%
